AutoGluon(AutoML) 사용해보기
AutoGluon Tabular - Quick Start : https://auto.gluon.ai/stable/tutorials/tabular/tabular-quick-start.html

AutoGluon Tabular - Essential Functionality : https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html

추가적으로 AutoGluon을 알고싶다면 AutoGluon Tabular - In Depth : https://auto.gluon.ai/stable/tutorials/tabular/tabular-indepth.html

Quick Start

In [1]:
#pip install autogluon

In [1]:
# Data import

from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd

df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# evaluate과 leaderboard함수에 valid 데이터를 넣어서 검증하기 위해 데이터 분할
train = df[:int(len(df) * 0.8)]
valid = df[int(len(df) * 0.8):]

train = TabularDataset(train.drop(['ID'], axis=1))
valid = TabularDataset(valid.drop(['ID'], axis=1))

# Training
# Define the predictor
predictor = TabularPredictor(label='Income', eval_metric='rmse')

# Training the model
predictor.fit(train_data=train, tuning_data=valid)

# Make predictions on test data
predictions = predictor.predict(test)

# Save predictions to a CSV file
predictions.to_csv('predictions.csv', index=False)


No path specified. Models will be saved in: "AutogluonModels/ag-20240407_111739"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240407_111739"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #44-Ubuntu SMP Fri Jan 19 20:16:38 UTC 2024
CPU Count

In [2]:

# Load the test data
test_data = TabularDataset(test.drop(['ID'], axis=1))

# Make predictions on the test data
predictions = predictor.predict(test_data)

# Add predictions to the test dataframe
test['predictions'] = predictions

# Save the test dataframe with predictions to a CSV file
#test.to_csv('test_with_predictions.csv', index=False)

In [36]:
#preds = model.predict(test_x)
submission = pd.read_csv('./sample_submission.csv')
submission['Income'] = predictions
submission.to_csv('./baseline_submission.csv', index=False)

In [54]:
predictions

0        23.832928
1        -8.404715
2       459.588898
3       583.369080
4         2.624141
           ...    
9995    932.753967
9996    704.264893
9997    394.666382
9998      6.769224
9999    517.470642
Name: Income, Length: 10000, dtype: float32

In [55]:

submission['Income'] = predictions
submission.to_csv('./baseline_submission.csv', index=False)
filtered_predictions = [i for i in predictions if i < 50]

In [56]:
filtered_predictions = {key: 0 if value < 100 else value for key, value in predictions.items()}
filtered_predictions
submission['Income'] = filtered_predictions
submission.to_csv('./lastsum.csv', index=False)

In [57]:
X_test = test.query('Age>=15').rename({'Working_Week (Yearly)':'Working_Week'},axis=1)\
.query('Employment_Status!="Not Working"').query('Working_Week!= 0 or Age<80')\
.query('Occupation_Status != "Unknown"')

In [58]:
X_test.index

Index([   2,    3,    5,    6,    9,   10,   11,   12,   13,   15,
       ...
       9987, 9988, 9990, 9992, 9993, 9994, 9995, 9996, 9997, 9999],
      dtype='int64', length=7673)

In [59]:
pred_series = pd.Series(submission['Income'], index=X_test.index)
pred_series

2        459.588898
3        583.369080
5        674.038269
6        454.338531
9       1048.528320
           ...     
9994     598.612854
9995     932.753967
9996     704.264893
9997     394.666382
9999     517.470642
Name: Income, Length: 7673, dtype: float64

In [60]:
submission['Income']

0         0.000000
1         0.000000
2       459.588898
3       583.369080
4         0.000000
           ...    
9995    932.753967
9996    704.264893
9997    394.666382
9998      0.000000
9999    517.470642
Name: Income, Length: 10000, dtype: float64

In [61]:
submission.loc[X_test.index.difference(X_test.index), 'Income'] = 0
submission.loc[X_test.index, 'Income'] = pred_series
submission

,ID,Income
0,TEST_0000,0.000000
1,TEST_0001,0.000000
2,TEST_0002,459.588898
3,TEST_0003,583.369080
4,TEST_0004,0.000000
...,...,...
9995,TEST_9995,932.753967
9996,TEST_9996,704.264893
9997,TEST_9997,394.666382
9998,TEST_9998,0.000000


In [62]:
submission.to_csv('submitgg.csv',index=False)